In [3]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('spark_hdfs_to_hdfs').getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("WARN")

In [14]:
MASTER_NODE_INSTANCE_NAME="cluster-a1c7-m"

In [15]:
log_files_rdd = sc.textFile('hdfs://{}/data/logs_example/*'.format(MASTER_NODE_INSTANCE_NAME))

In [16]:
splitted_rdd = log_files_rdd.map(lambda x: x.split(" "))
selected_col_rdd = splitted_rdd.map(lambda x: (x[0], x[3], x[5], x[6]))

In [18]:
columns = ["ip","date","method","url"]
logs_df = selected_col_rdd.toDF(columns)
logs_df.createOrReplaceTempView('logs_df')
print(type(logs_df),dir(logs_df))

<class 'pyspark.sql.dataframe.DataFrame'> ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_collectAsArrow', '_jcols', '_jdf', '_jmap', '_jseq', '_lazy_rdd', '_repr_html_', '_sc', '_schema', '_sort_cols', '_support_repr_html', 'agg', 'alias', 'approxQuantile', 'cache', 'checkpoint', 'coalesce', 'colRegex', 'collect', 'columns', 'corr', 'count', 'cov', 'createGlobalTempView', 'createOrReplaceGlobalTempView', 'createOrReplaceTempView', 'createTempView', 'crossJoin', 'crosstab', 'cube', 'describe', 'distinct', 'drop', 'dropDuplicates', 'drop_duplicates', 'dropna', 'dtypes', 'exceptAll', 'explain', 'fillna', 'filter', 'first', 'foreach', 'foreachPartition', 'freqItems'

In [25]:
sql = f"""
  SELECT
  url,
  count(*) as count
  FROM logs_df
  WHERE url LIKE '%/article%'
  GROUP BY url
  """
article_count_df = spark.sql(sql)
print(" ### Get only articles and blogs records ### ")
article_count_df.show(5)

 ### Get only articles and blogs records ### 
+--------------------+-----+
|                 url|count|
+--------------------+-----+
|/blog/articles/we...|    1|
|/articles/openlda...|   18|
|/articles/week-of...|   11|
|        /articles/dy|    1|
|/articles/ssh-sec...|   55|
+--------------------+-----+
only showing top 5 rows



In [27]:
article_count_df.describe()

DataFrame[summary: string, url: string, count: string]

In [30]:
article_count_df.write.save('hdfs://{}/data/article_count_df'.format(MASTER_NODE_INSTANCE_NAME), format='csv', mode='overwrite')